In [1]:
import numpy as np
import cs_vqe as c
from copy import deepcopy as copy

import ast
import matplotlib.pyplot as plt
import os

In [2]:
import tqdm
from tqdm.notebook import tqdm
import Graph as graph


In [3]:
# working_dir = os.path.dirname(os.path.abspath(__file__)) # gets directory where running python file is!
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [4]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [5]:
data_csvqe_results_file = os.path.join(data_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

In [6]:
# # fn_form =hamiltonians['H1-O1_STO-3G_singlet'][5][4]
# # all_mappings = hamiltonians['H1-O1_STO-3G_singlet'][5][3]
# q_vec = hamiltonians['H1-O1_STO-3G_singlet'][5][1]
# r_vec = hamiltonians['H1-O1_STO-3G_singlet'][5][2]
# hamiltonians['H1-O1_STO-3G_singlet'][5][1]

# Convert into Openfermion format

In [7]:
import conversion_scripts as conv_scr

In [8]:
updated_hamiltonians={}
for key in csvqe_results.keys(): # <- note using csvqe_results (has less molecules than hamiltonians.keys())
    

    ham_noncon = hamiltonians[key][3]
    ep_state = [list(hamiltonians[key][5][1]), list(hamiltonians[key][5][2])] # ground state of noncontextual Ham
    ham = hamiltonians[key][2]

    model = c.quasi_model(ham_noncon)
    fn_form = c.energy_function_form(ham_noncon, model)
    
#     _, _, _, ham_out, ham_noncon_out = c.diagonalize_G(model,fn_form,ep_state, ham, ham_noncon)

    N_qubits = hamiltonians[key][1]
    order= csvqe_results[key][-1] # <-- order specified from greedy run by Will
    qubit_removal_order=copy(order)
    
    reduced_Con_hamiltonians = c.get_reduced_hamiltonians(ham ,model, fn_form, ep_state, order)
    
    
    Contextual_Hamiltonian_list=[]
    qNo_list=[]
    for qNo, Ham in enumerate(reduced_Con_hamiltonians):            
#         if c.contextualQ_ham(Ham) is False:
#             raise ValueError('have not found contextual H')
        Ham_Openf = conv_scr.Get_Openfermion_Hamiltonian(Ham)
        Contextual_Hamiltonian_list.append(list(Ham_Openf))
        qNo_list.append(qNo)
    
    updated_hamiltonians[key] = {
                                'encoding':hamiltonians[key][0],
                                'n_qubits': hamiltonians[key][1],
                                'full_H': conv_scr.Get_Openfermion_Hamiltonian(hamiltonians[key][2]),
                                'nonC_H_greedyDFS': conv_scr.Get_Openfermion_Hamiltonian(hamiltonians[key][3]),
                                'FCI':hamiltonians[key][4],                         
                                'gstate_noncon':hamiltonians[key][5], 
                                'Contextual_Hamiltonian_list': Contextual_Hamiltonian_list, #ham_out,   
                                'Contextual_Hamiltonian_qubitNo_list': qNo_list,  
                                'non_Contextual_H': conv_scr.Get_Openfermion_Hamiltonian(ham_noncon),# ham_noncon_out, 
                                'qubit_removal_order':qubit_removal_order
                             }

In [9]:
print(updated_hamiltonians['Mg1_STO-3G_singlet']['qubit_removal_order'])
# updated_hamiltonians['Mg1_STO-3G_singlet']['Contextual_Hamiltonian_list']

[0, 1, 2, 9, 3, 12, 4, 5, 6, 7, 8, 10, 11]


In [10]:
# checking keys match!
for key in updated_hamiltonians.keys():
    if key not in csvqe_results.keys():
        raise ValueError(f'{key} not in csvqe_results')

In [11]:
import tqdm
from tqdm.notebook import tqdm
from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian

In [12]:
unitary_paritioning_of_Con_H={}

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'


for key in tqdm(list(updated_hamiltonians.keys())):
    
    H_con_List = updated_hamiltonians[key]['Contextual_Hamiltonian_list']
    
    
    anti_commuting_different_red_list=[]
    for i, H_con in enumerate(H_con_List):
        n_qubits = updated_hamiltonians[key]['Contextual_Hamiltonian_qubitNo_list'][i]
        anti_commuting_sets = Clique_cover_Hamiltonian(H_con, 
                                                             n_qubits, 
                                                             commutativity_flag, 
                                                             Graph_colouring_strategy)
        
        anti_commuting_different_red_list.append(anti_commuting_sets)
    
    H_con_reduced_and_antiC_set = list(zip(H_con_List, anti_commuting_different_red_list))
    
    unitary_paritioning_of_Con_H[key] = H_con_reduced_and_antiC_set

  0%|          | 0/26 [00:00<?, ?it/s]

In [13]:
for key in unitary_paritioning_of_Con_H:
    
    H_con_and_antiC_set_list_different_reductions = unitary_paritioning_of_Con_H[key]
    
    for H_con, AC in H_con_and_antiC_set_list_different_reductions:
        AC = len(AC)
        standard=len(list(H_con))
        print(f'AC : {AC: <6} vs standard: {standard: >6}')
    
    print('\n')

AC : 1      vs standard:      1
AC : 2      vs standard:      3
AC : 4      vs standard:     10
AC : 13     vs standard:     35


AC : 1      vs standard:      1
AC : 2      vs standard:      3
AC : 5      vs standard:     10
AC : 9      vs standard:     20
AC : 20     vs standard:     54
AC : 32     vs standard:    102


AC : 1      vs standard:      1
AC : 2      vs standard:      3
AC : 4      vs standard:      8
AC : 13     vs standard:     34
AC : 19     vs standard:     62
AC : 26     vs standard:    122


AC : 1      vs standard:      1
AC : 2      vs standard:      3
AC : 4      vs standard:      8
AC : 13     vs standard:     34
AC : 19     vs standard:     62
AC : 26     vs standard:    122


AC : 1      vs standard:      1
AC : 2      vs standard:      3
AC : 4      vs standard:     10
AC : 12     vs standard:     34
AC : 21     vs standard:     84
AC : 43     vs standard:    155
AC : 50     vs standard:    319


AC : 1      vs standard:      1
AC : 2      vs standard:      

In [14]:
import pickle
import datetime

In [15]:
unique_file_id = datetime.datetime.now().strftime('%Y%b%d-%H%M%S%f')

In [16]:
output_dir = os.path.join(working_dir, 'Pickle_out')

file_name1 = 'EXP_ordering_anticommuting_vs_standard_conH_{}.pickle'.format(unique_file_id)

file_out1=os.path.join(output_dir, file_name1)
with open(file_out1, 'wb') as outfile:
    pickle.dump(unitary_paritioning_of_Con_H, outfile)

file_name2 = 'EXP_ordering_updated_hamiltonians_{}.pickle'.format(unique_file_id)
file_out2=os.path.join(output_dir, file_name2)
with open(file_out2, 'wb') as outfile:
    pickle.dump(updated_hamiltonians, outfile)

In [17]:
## test if pickle worked
file_name = 'EXP_ordering_anticommuting_vs_standard_conH.pickle'
with open(file_name, 'rb') as infile:
    loaded_dict = pickle.load(infile)
    


FileNotFoundError: [Errno 2] No such file or directory: 'EXP_ordering_anticommuting_vs_standard_conH.pickle'

In [ ]:
updated_hamiltonians['H3_STO-3G_singlet_1+']

# Sanity check
- check the overall energy to see if correct!